Reading and pre-processing of time-series data for 4 stations in Delhi

Datset has 13 Columns out of which 12 are pollutants like PM2.5, PM10, NO, etc.
Please find the Data from data_new folder. 


In [ ]:
from math import sqrt
import numpy as np
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from pandas import read_csv
from datetime import datetime
from keras.layers import Bidirectional
from keras.layers import Dropout
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import seaborn as sn
import copy


# load data
def parse(x):
	return datetime.strptime(x, '%Y %m %d %H')

# Anand Vihar 
dfav = read_csv('data_new/anand_vihar_jan19_dec20_8h.csv')
dfav.columns = ['From Date','PM10','NO','NO2','WS','CO','Benzene','NOx','Ozone','SO2','NH3','Toluene','PM2.5']

# Bawana
dfbw= read_csv('data_new/Bawana_jan19-dec20_8h.csv')
dfbw.columns = ['From Date','PM10','NO','NO2','WS','CO','Benzene','NOx','Ozone','SO2','NH3','Toluene','PM2.5']

# Vivek Vihar
dfvv = read_csv('data_new/vivek_vihar_jan19_dec20_8h.csv')
dfvv.columns = ['From Date','PM10','NO','NO2','WS','CO','Benzene','NOx','Ozone','SO2','NH3','Toluene','PM2.5']


dfwz = read_csv('data_new/Wazirpur_8h_jan19-dec20.csv')
dfwz.columns = ['From Date','PM10','NO','NO2','WS','CO','Benzene','NOx','Ozone','SO2','NH3','Toluene','PM2.5']


# DTU
dfdt = read_csv('data_new/DTU_8h_jan19-dec20.csv')
dfdt.columns = ['From Date','PM10','NO','NO2','WS','CO','Benzene','NOx','Ozone','SO2','NH3','Toluene','PM2.5']



# Edit it accordingly for 4 different stations: Currently set-up for Anand Vihar
df= dfav

print("Shape of df:",df.shape)


print(df.head(5))


df = df.replace(to_replace='None', value=np.nan)
df = df.dropna(axis = 0, how = 'all')
df['PM10'].fillna(df['PM10'].median(), inplace=True)
df['NO'].fillna(df['NO'].median(), inplace=True)
df['NO2'].fillna(df['NO2'].median(), inplace=True)
df['WS'].fillna(df['WS'].median(), inplace=True)
df['CO'].fillna(df['CO'].median(), inplace=True)
df['Benzene'].fillna(df['Benzene'].median(), inplace=True)
df['NOx'].fillna(df['NOx'].median(), inplace=True)
df['Ozone'].fillna(df['Ozone'].median(), inplace=True)
df['SO2'].fillna(df['SO2'].median(), inplace=True)
df['NH3'].fillna(df['NH3'].median(), inplace=True)
df['Toluene'].fillna(df['Toluene'].median(), inplace=True)
df['PM2.5'].fillna(df['PM2.5'].median(), inplace=True)

print(" \nCount total NaN at each column in a DataFrame : \n\n", df.isnull().sum())


In [ ]:
df.head()

In [ ]:
features = df.loc[:, df.columns != 'From Date']
features = features.astype(float)
features.head()

The following code computes statistical measures (minimum, maximum, mean, and standard deviation) for various air quality features. These features include PM10, NO, NO2, WS (wind speed), CO, Benzene, NOx, Ozone, SO2, NH3, Toluene, and PM2.5.

In [ ]:
# Table for range, mean and std for different features
feat = features['PM10'].values
print("PM10 :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['NO'].values
print("NO :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['NO2'].values
print("NO2 :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['WS'].values
print("WS :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['CO'].values
print("CO :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['Benzene'].values
print("Benzene :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['NOx'].values
print("NOx :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['Ozone'].values
print("Ozone :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['SO2'].values
print("SO2 :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['NH3'].values
print("NH3 :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['Toluene'].values
print("Toluene :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))
feat = features['PM2.5'].values
print("PM2.5 :", "Min = ",min(feat), ", Max = ", max(feat) , ", Mean = ", np.mean(feat), ", Std = ",np.std(feat))

Correlation Matrix and Heatmap for features

In [ ]:

corr_matrix = features.corr()
f, ax = plt.subplots(figsize=(11, 15))
heatmap = sn.heatmap(corr_matrix,
                      #mask = mask,
                      square = True,
                      linewidths = .5,
                      cmap = 'coolwarm',
                      cbar_kws = {'shrink': .4,
                                'ticks' : [-1, -.5, 0, 0.5, 1]},
                      vmin = -1,
                      vmax = 1,
                      annot = True,
                      annot_kws = {"size": 12})

#add the column names as labels
ax.set_yticklabels(corr_matrix.columns, rotation = 0)
ax.set_xticklabels(corr_matrix.columns)

sn.set_style({'xtick.bottom': True}, {'ytick.left': True})

Preparation of dataframe for analysis

In [ ]:
df['Date'] = pd.to_datetime(df['From Date']) 
df['year'] = df['Date'].dt.year 
df['month'] = df['Date'].dt.month 
df['day'] = df['Date'].dt.day 
df['hour'] = df['Date'].dt.hour 
df['minute'] = df['Date'].dt.minute


In [ ]:
df.head()

Splitting into train/test for different LSTM models.

In [ ]:
df_train = df[(df["year"] <2020) | (df['month']<=5)]

df_test=df[(df["year"] >2019) & (df['month']<=12) & (df['month']>5)]
df.to_csv('pollution.csv')
df_train.to_csv('train_pollution.csv')
df_test.to_csv('test_pollution.csv')

For season-wise training and testing. Uncomment this for corresponding seasonal analysis.

In [ ]:
## For season-wise training and testing
"""
df_train = df[((df["year"] == 2019) ) & (df['month'] >= 2) & (df['month'] <=9)]
df_test=df[(df["year"]==2020) & (df['month'] >= 2) & (df['month'] <=9)]
df.to_csv('pollution.csv')
df_train.to_csv('train_pollution.csv')
df_test.to_csv('test_pollution.csv')"""

In [ ]:
# Note: df_july has to be used only for seasonal analysis
df_july = df[((df["year"] == 2020) ) & (df['month'] == 9)  ]
#df_july = df[((df["year"] == 2020) ) & (df['month'] >= 8) & (df['month'] <=9)]
df_july.to_csv('july_test.csv')

Plotting the time-series data for train

In [ ]:
x_val = np.linspace(1,df_train.shape[0],num = df_train.shape[0])
plot_list = np.array(df_train['PM2.5'].values)
y_val = np.array([(float)(k) for k in plot_list])
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111) 
plt.tick_params(labelsize=25)
params = {'legend.fontsize': 25, 'legend.handlelength': 2}
plt.rcParams.update(params)
surrogate_plot = ax.plot(x_val,y_val,linestyle='-', linewidth= 1, color= 'b', label= 'PM2.5')
ax.set_xlabel('Time (Samples)',size= 25)
ax.set_ylabel('PM2.5 Value', size= 25)
ax.set_xlim([0,np.amax(x_val)])   
ax.legend(loc='best')
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()
plt.clf()

Plotting the time-series data for test

In [ ]:
x_val = np.linspace(1,df_test.shape[0],num = df_test.shape[0])
plot_list = np.array(df_test['PM2.5'].values)
y_val = np.array([(float)(k) for k in plot_list])
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111) 
plt.tick_params(labelsize=25)
params = {'legend.fontsize': 25, 'legend.handlelength': 2}
plt.rcParams.update(params)
surrogate_plot = ax.plot(x_val,y_val,linestyle='-', linewidth= 1, color= 'r', label= 'PM2.5')
ax.set_xlabel('Time Steps',size= 25)
ax.set_ylabel('PM2.5 Concentration', size= 25)
ax.set_xlim([0,np.amax(x_val)])   
ax.legend(loc='best')
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()
plt.clf()

In [ ]:
dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,1:-6]
values_test=values_test[:,1:-6]
print(values_train.shape,values_test.shape)
n_hours=1
n_out= 1
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
print(values.shape)


Defining different models and trial experiments

In [ ]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
import datetime

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)






def MODEL_FNN_adam(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    print(train_X.shape)
    train_X=np.reshape(train_X,(train_X.shape[0],n_hours*n_features))
    test_X=np.reshape(test_X,(test_X.shape[0],n_hours*n_features))
    val_X=np.reshape(val_X,(val_X.shape[0],n_hours*n_features))

    model = Sequential()
    model.add(Dense(64, input_dim=train_X.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_out))
    model.compile(loss='mse', optimizer='adam')
    # fit the keras model on the dataset
    model.fit(train_X, train_y, epochs=200, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2



def MODEL_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_hours,n_features)))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
# fit model
    model.fit(train_X, train_y, epochs=200, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2

def MODEL_Bi_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu',input_shape=(n_hours,n_features))))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
# fit model
    # Important note : epochs = 50 for seasonal model
    model.fit(train_X, train_y, epochs=200, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2


def MODEL_EN_DC(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))
    val_y = val_y.reshape((val_y.shape[0], val_y.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, activation='relu',input_shape=(n_hours,n_features)))
    model.add(RepeatVector(n_out))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
# fit model
    model.fit(train_X, train_y, epochs=200, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2

           
            
            
            
n_out_max,n_hour_max= 10,5
out_range=[10]
dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,1:-6]
values_test=values_test[:,1:-6]

values_train = values_train.astype('float32')
values_test = values_test.astype('float32')

Data preparation for training/testing of models

In [ ]:
# Run this cell for general analysis of models
dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,1:-6]
values_test=values_test[:,1:-6]

values_train = values_train.astype('float32')
values_test = values_test.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
concat = np.concatenate((values_train,values_test),axis = 0)
maxv = max(concat[:,-1])
minv = min(concat[:,-1])
scaled = scaler.fit_transform(concat)
scaled_train = scaled[:values_train.shape[0],:]
scaled_test = scaled[values_train.shape[0]:,:]

Data preparation here is only for 1 month comparison for September using SBD-LSTM


In [ ]:
# Run this cell only for seasonal analysis of September
data_july = read_csv('july_test.csv', header=0, index_col=0)
values_july = data_july.values
values_july = values_july[:,1:-6]
values_july = values_july.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
concat = np.concatenate((values_train,values_test,values_july),axis = 0)
maxv = max(concat[:,-1])
minv = min(concat[:,-1])
scaled = scaler.fit_transform(concat)
scaled_train = scaled[:values_train.shape[0],:]
scaled_test = scaled[values_train.shape[0]: (values_train.shape[0] + values_test.shape[0]),:]
scaled_july = scaled[(values_train.shape[0] + values_test.shape[0]):,:]

Training the FNN model using Adam

Feedforward Neural Network (FNN):
* A feedforward neural network architecture.
* Suitable for regression tasks like time series forecasting.

In [ ]:
time1 = time.time()
outfile_adam = open('results/AnandVihar/results_adam.txt','a+')
train_adam = open('results/AnandVihar/adam_train.txt','a+')
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            err , err2 = MODEL_FNN_adam(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array(err)
            allres2 = np.array(err2)
            for item in allres:
              item = round(item,4)
              outfile_adam.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_adam.write(str(item) + "  ")  
            #np.savetxt(outfile_adam,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_adam,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_adam.write("\n")    
        train_adam.write("\n")   

outfile_adam.close()
train_adam.close()
time2 = time.time()        
print("MODEL_FNN_adam:",time2-time1)


Training LSTM model

Long Short-Term Memory (LSTM):
* A type of recurrent neural network (RNN).
* Designed for sequence modeling and time-series data.
* Captures long-range dependencies using memory cells.

In [ ]:
time1 = time.time()
outfile_lstm = open('results/AnandVihar/results_lstm.txt','a+')
train_lstm = open('results/AnandVihar/lstm_train.txt','a+')
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            err , err2=MODEL_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array(err)
            allres2 = np.array(err2)
            for item in allres:
              item = round(item,4)
              outfile_lstm.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_lstm.write(str(item) + "  ")
            #np.savetxt(outfile_lstm,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_lstm,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_lstm.write("\n")    
        train_lstm.write("\n") 

outfile_lstm.close()    
train_lstm.close()             
time2 = time.time()        
print("MODEL_LSTM:",time2-time1)


Training Bi-LSTM model

> Note: For seasonal model the split interval for validation split will be after 365 data points.

Bidirectional LSTM (Bi-LSTM):
* Processes input sequences in both forward and backward directions.
* Enhances context capture by considering past and future information.

In [ ]:
outfile_bi_lstm = open('results/AnandVihar/results_bi_lstm.txt','a+')
train_bi_lstm = open('results/AnandVihar/bi_lstm_train.txt','a+')
time1 = time.time()
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            """
            Uncomment below two lines for Seasonal Analysis and comment above two lines
            """
            #test_X, test_y = val_X[365:,:,:],val_y[365:,:]
            #val_X, val_y = val_X[0:365,:,:],val_y[0:365,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            err , err2=MODEL_Bi_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array(err)
            allres2 = np.array(err2)
            for item in allres:
              item = round(item,4)
              outfile_bi_lstm.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_bi_lstm.write(str(item) + "  ")
            #np.savetxt(outfile_bi_lstm,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_bi_lstm,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_bi_lstm.write("\n")    
        train_bi_lstm.write("\n") 
            
outfile_bi_lstm.close()    
train_bi_lstm.close()    
time2 = time.time()
print("MODEL_Bi_LSTM:",time2-time1)


Training Encoder-Decoder LSTM model

Encoder-Decoder LSTM:
* Consists of an encoder and a decoder (two LSTM networks).
* Encoder compresses input sequences into a context vector.
* Decoder generates output sequences based on the context vector.

In [ ]:
outfile_en_dc = open('results/AnandVihar/results_en_dc.txt','a+')
train_en_dc = open('results/AnandVihar/en_dc_train.txt','a+')
time1 = time.time()
out_range=[10]
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            err,err2 = MODEL_EN_DC(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array([item[0] for item in err])
            allres2 = np.array([item[0] for item in err2])
            for item in allres:
              item = round(item,4)
              outfile_en_dc.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_en_dc.write(str(item) + "  ")
            #np.savetxt(outfile_en_dc,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_en_dc,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_en_dc.write("\n")    
        train_en_dc.write("\n") 
            
outfile_en_dc.close()    
train_en_dc.close()        
time2 = time.time()
print("MODEL_ENC_DCD:",time2-time1)


Plotting different bars:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
def show_results(r,method):
    x = datetime.datetime.now()
    print("r = ", r)
    u=np.mean(r,axis=0)
    sigma=np.std(r,axis=0)
    lower_bar= u-1.96*(sigma/(np.sqrt(1)))
    upper_bar= u+1.96*(sigma/(np.sqrt(1)))
    barWidth = 0.3
    bars1=u
    rl= np.arange(len(bars1))+1
    plt.bar(rl, bars1, width = barWidth, color = 'red', edgecolor = 'black', yerr=1.96*(sigma/(np.sqrt(r.shape[0]))), capsize=7, label=method+' with using 10 multivariate steps for 30 iterations')
    plt.ylabel('mean of normalized RMSE for 30 iterations')
    plt.xlabel('Number of steps')
    plt.legend()
    plt.show()
    

In [ ]:

ladam = []
# change file directory for different stations accordingly
file1 = open('results/AnandVihar/results_adam.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  ladam.append([float(li[k]) for k in range(10)])

dADAM=np.array(ladam)
print(dADAM.shape)
show_results(dADAM,'fnn_ADAM')


In [ ]:

dLSTM = []
file1 = open('results/AnandVihar/results_lstm.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dLSTM.append([float(li[k]) for k in range(10)])
  
dLSTM=np.array(dLSTM)
print(dLSTM.shape)

show_results(dLSTM,'LSTM')

In [ ]:
dEN = []
file1 = open('results/AnandVihar/results_en_dc.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dEN.append([float(li[k]) for k in range(10)])
dEN=np.array(dEN)
print(dEN.shape)
show_results(dEN,'Encoder-Decoder')

In [ ]:
dBI = []
file1 = open('results/AnandVihar/results_bi_lstm.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dBI.append([float(li[k]) for k in range(10)])
dBI=np.array(dBI)
print(dBI.shape)
show_results(dBI,'Bi directional LSTM')

In [ ]:
def show_results_Combined(r1,r2,r3,r4):
    x = datetime.datetime.now()
    print(r1.shape)
    u1=np.mean(r1,axis=0)
    u2=np.mean(r2,axis=0)
    u3=np.mean(r3,axis=0)
    u4=np.mean(r4,axis=0)
    sigma1=np.std(r1,axis=0)
    sigma2=np.std(r2,axis=0)
    sigma3=np.std(r3,axis=0)
    sigma4=np.std(r4,axis=0)
    barWidth = 0.17
    bars1=u1
    bars2=u2
    bars3=u3
    bars4=u4
    ra1= np.arange(len(bars1))+1
    ra2 = [y + barWidth for y in ra1]
    ra3 = [y + barWidth for y in ra2]
    ra4 = [y + barWidth for y in ra3]
    plt.bar(ra1, bars1, width = barWidth, color = 'blue', edgecolor = 'black', yerr=1.96*(sigma1/(np.sqrt(r1.shape[0]))), capsize=7, label='FNN-ADAM')
    plt.bar(ra2, bars2, width = barWidth, color = 'yellow', edgecolor = 'black', yerr=1.96*(sigma2/(np.sqrt(r2.shape[0]))), capsize=7, label='LSTM')
    plt.bar(ra3, bars3, width = barWidth, color = 'green', edgecolor = 'black', yerr=1.96*(sigma3/(np.sqrt(r3.shape[0]))), capsize=7, label='BD-LSTM')
    plt.bar(ra4, bars4, width = barWidth, color = 'red', edgecolor = 'black', yerr=1.96*(sigma4/(np.sqrt(r4.shape[0]))), capsize=7, label='ED-LSTM')
    plt.ylabel('mean of RMSE for 30 iterations')
    plt.xlabel('Number of steps')
    plt.legend()
    plt.show()
show_results_Combined(dADAM,dLSTM,dBI,dEN)
## Change the fontsize of legends according to convenience

In [ ]:
import plotly.graph_objects as go

u1=np.mean(dADAM,axis=0)
u2=np.mean(dLSTM,axis=0)
u3=np.mean(dEN,axis=0)
u4=np.mean(dBI,axis=0)
sigma1=np.array([1.96*(np.std(dADAM[:,k])/(np.sqrt(dADAM[:,k].shape[0]))) for k in range(10)])
sigma2=np.array([1.96*(np.std(dLSTM[:,k])/(np.sqrt(dLSTM[:,k].shape[0]))) for k in range(10)])
sigma3=np.array([1.96*(np.std(dEN[:,k])/(np.sqrt(dEN[:,k].shape[0]))) for k in range(10)])
sigma4=np.array([1.96*(np.std(dBI[:,k])/(np.sqrt(dBI[:,k].shape[0]))) for k in range(10)])
fig = go.Figure()
df=pd.DataFrame()
df['ADAM']=u1
df['LSTM']=u2
df['EN']=u3
df['BI']=u4
df['steps']=[1,2,3,4,5,6,7,8,9,10]
fig.add_trace(go.Bar(x=df['steps'],
                y=df['ADAM'],
                     error_y=dict(type='data', array=sigma1),
                name='FNN-ADAM',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df['steps'],
                y=df['LSTM'],
                error_y=dict(type='data', array=sigma2),
                name='LSTM',
                marker_color='rgb(200,125,150)'
                ))
fig.add_trace(go.Bar(x=df['steps'],
                y=df['EN'],
                error_y=dict(type='data', array=sigma3),
                name='ED-LSTM',
                marker_color='rgb(13, 255, 20)'
                ))
fig.add_trace(go.Bar(x=df['steps'],
                y=df['BI'],
                error_y=dict(type='data', array=sigma4),
                name='BD-LSTM',
                marker_color='rgb(255, 10, 10)'
                ))

fig.update_layout(
    #title='mean of RMSE for 30 iterations' ,
    width = 800,
    xaxis_tickfont_size=20,
    xaxis=dict(
        title='Step Size',
        titlefont_size=20,
        tickfont_size=20,
    ),
    yaxis=dict(
        title='RMSE(Mean)',
        titlefont_size=20,
        tickfont_size=20,
    ),
    legend=dict(
        x=0,
        y=1.0,
        font=dict(
            size=18
        ),
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.25, # gap between bars of adjacent location coordinates.
    bargroupgap=0.05# gap between bars of the same location coordinate.
   
    
)
fig.show()

Data Preparation 1: Random Shuffling

The training data is randomly shuffled to enhance model performance.

In [ ]:

def Shuffled_BiLSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_hours,n_features))))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
# fit model
    model.fit(train_X, train_y, epochs=64, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle= True)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2


time1 = time.time()
outfile_bilstm_shuffled = open('results/AnandVihar/results_bilstm_shuffled.txt','a+')
train_shuffled_bilstm = open('results/AnandVihar/bilstm_shuffled_train.txt','a+')
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            print("n_features = ",n_features)
            err , err2= Shuffled_BiLSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array(err)
            allres2 = np.array(err2)
            for item in allres:
              item = round(item,4)
              outfile_bilstm_shuffled.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_shuffled_bilstm.write(str(item) + "  ")
            #np.savetxt(outfile_bilstm_shuffled,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_shuffled_bilstm,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_bilstm_shuffled.write("\n")    
        train_shuffled_bilstm.write("\n")     

outfile_bilstm_shuffled.close()    
train_shuffled_bilstm.close()            
time2 = time.time()        
print("MODEL_time:",time2-time1)
#np.savetxt("10i_"+str(result.shape[1])+"o_FNN_adam_"+str(result.shape[0])+"i.csv"+str(datetime.datetime.now()), result, delimiter=",")


Data Preparation 2: Univariate vs Multivariate Analysis using our best model- Bi-LSTM

In [ ]:

def split_sequences_univariate(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        
        seq_x, seq_y = sequences[i:end_ix], sequences[end_ix-1:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def MODEL_Univariate_Bi_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_hours,n_features))))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
    model.fit(train_X, train_y, epochs=1000, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)
    yhat11 = model.predict(test_X, verbose=0)
    yhat11_inv = ((maxv - minv) * yhat11  + minv)
    test_y_inv = ((maxv - minv) * test_y  + minv)
    print("Predicted = ", yhat11_inv)
    print("Actual = ", test_y_inv)
    err = np.sqrt(((np.sum((yhat11_inv-test_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step is =",err )
    yhat12 = model.predict(train_X, verbose = 0)
    yhat12_inv = ((maxv - minv) * yhat12  + minv)
    train_y_inv = ((maxv - minv) * train_y  + minv)
    err2 = np.sqrt(((np.sum((yhat12_inv-train_y_inv)**2,axis=0))))
    print("RMSE error using ",n_hours," state size to predict ",n_out," step for train is =",err2 )
    return err , err2



dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,-7]
values_test=values_test[:,-7]
values_train = values_train.astype('float32')
values_test = values_test.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
concat = np.concatenate((values_train,values_test),axis = 0)
print(concat.shape)
maxv = max(concat)
minv = min(concat)
scaled = (concat-minv)/(maxv-minv)
scaled_train = scaled[:values_train.shape[0]]
scaled_test = scaled[values_train.shape[0]:]
outfile_bi_lstm = open('results/AnandVihar/results_bi_lstm_univariate.txt','a+')
train_bi_lstm = open('results/AnandVihar/bi_lstm_univarate_train.txt','a+')
time1 = time.time()
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences_univariate(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences_univariate(scaled_test,n_hours,n_out)
            val_X = val_X.reshape(val_X.shape[0] , val_X.shape[1] , 1)
            train_X = train_X.reshape(train_X.shape[0] , train_X.shape[1] , 1)
            print(val_X.shape,val_y.shape)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = 1
            err , err2=MODEL_Univariate_Bi_LSTM(train_X,test_X,train_y,test_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            allres = np.array(err)
            allres2 = np.array(err2)
            for item in allres:
              item = round(item,4)
              outfile_bi_lstm.write(str(item) + "  ")
            for item in allres2:
              item = round(item,4)
              train_bi_lstm.write(str(item) + "  ")
            #np.savetxt(outfile_bi_lstm,  allres  , fmt='%1.4f', newline=' ' )
            #np.savetxt(train_bi_lstm,  allres2  , fmt='%1.4f', newline=' ' )
        outfile_bi_lstm.write("\n")    
        train_bi_lstm.write("\n") 

outfile_bi_lstm.close()    
train_bi_lstm.close()            
        
time2 = time.time()
print("MODEL_Bi_LSTM:",time2-time1)

In [ ]:
shuffled = []
file1 = open('results/AnandVihar/results_bilstm_shuffled.txt','r')
#file1 = open('results/AnandVihar/1_shuffled_test.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  shuffled.append([float(li[k]) for k in range(10)])
shuffled = np.array(shuffled)
flat_shuffled = np.ravel(shuffled , order = 'F')
print("Mean = ", np.mean(flat_shuffled))
print("Error = ", 1.96*(np.std(flat_shuffled)/(np.sqrt(flat_shuffled.shape[0]))))

Plotting for results of different models

In [ ]:
##Get the test results
#FNN-ADAM
dADAM_train = []
file1 = open('results/AnandVihar/adam_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dADAM_train.append([float(li[k]) for k in range(10)])
dADAM_train=np.array(dADAM_train)

#LSTM
dLSTM_train = []
file1 = open('results/AnandVihar/lstm_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dLSTM_train.append([float(li[k]) for k in range(10)])
dLSTM_train=np.array(dLSTM_train)

#ED-LSTM
dEN_train = []
file1 = open('results/AnandVihar/en_dc_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dEN_train.append([float(li[k]) for k in range(10)])
dEN_train=np.array(dEN_train)

#BD-LSTM
dBI_train = []
file1 = open('results/AnandVihar/bi_lstm_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dBI_train.append([float(li[k]) for k in range(10)])
dBI_train=np.array(dBI_train)

#BD-LSTM*
shuffled_train = []
file1 = open('results/AnandVihar/bilstm_shuffled_train.txt','r')
#file1 = open('results/AnandVihar/1_shuffled_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  shuffled_train.append([float(li[k]) for k in range(10)])
shuffled_train=np.array(shuffled_train)

#UBD-LSTM
UBD_train = []
file1 = open('results/AnandVihar/bi_lstm_univarate_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  UBD_train.append([float(li[k]) for k in range(10)])
UBD_train=np.array(UBD_train)
#SBD-LSTM
SBD_train = []
file1 = open('results/AnandVihar/seasonal_bi_lstm_train.txt','r')
#file1 = open('results/AnandVihar/1_seasonal_train.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  SBD_train.append([float(li[k]) for k in range(10)])
SBD_train=np.array(SBD_train)

In [ ]:
## Corresponding Test RMSE with Mean and Error across 10 horizons
#flat_dSGD = np.ravel(dSGD , order = 'F')
flat_dADAM = np.ravel(dADAM , order = 'F')
flat_dLSTM = np.ravel(dLSTM , order = 'F')
flat_dEN = np.ravel(dEN , order = 'F')
flat_dBI = np.ravel(dBI , order = 'F')

dBI_univ = []
file1 = open('results/AnandVihar/results_bi_lstm_univariate.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dBI_univ.append([float(li[k]) for k in range(10)])
dBI_univ = np.array(dBI_univ)
flat_dBI_univ = np.ravel(dBI_univ , order = 'F')

dBI_season = []
file1 = open('results/AnandVihar/result_seasonal_bi_lstm.txt','r')
#file1 = open('results/AnandVihar/1_seasonal_test.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:10]
  dBI_season.append([float(li[k]) for k in range(10)])
dBI_season = np.array(dBI_season)
flat_dBI_season = np.ravel(dBI_season , order = 'F')

#m1 = np.mean(flat_dSGD)
m2 = np.mean(flat_dADAM)
m3 = np.mean(flat_dLSTM)
m4 = np.mean(flat_dEN)
m5 = np.mean(flat_dBI)
m6 = np.mean(flat_dBI_univ)
m7 = np.mean(flat_shuffled)
m8 = np.mean(flat_dBI_season)

#s1 = 1.96*(np.std(flat_dSGD)/(np.sqrt(flat_dSGD.shape[0])))
s2 = 1.96*(np.std(flat_dADAM)/(np.sqrt(flat_dADAM.shape[0])))
s3 = 1.96*(np.std(flat_dLSTM)/(np.sqrt(flat_dLSTM.shape[0])))
s4 = 1.96*(np.std(flat_dEN)/(np.sqrt(flat_dEN.shape[0])))
s5 = 1.96*(np.std(flat_dBI)/(np.sqrt(flat_dBI.shape[0])))
s6 = 1.96*(np.std(flat_dBI_univ)/(np.sqrt(flat_dBI_univ.shape[0])))
s7 = 1.96*(np.std(flat_shuffled)/(np.sqrt(flat_shuffled.shape[0])))
s8 = 1.96*(np.std(flat_dBI_season)/(np.sqrt(flat_dBI_season.shape[0])))

## Corresponding Train RMSE with Mean and Error across 10 horizons
flat_dADAM_train = np.ravel(dADAM_train , order = 'F')
flat_dLSTM_train = np.ravel(dLSTM_train , order = 'F')
flat_dEN_train = np.ravel(dEN_train , order = 'F')
flat_dBI_train = np.ravel(dBI_train , order = 'F')
flat_shuffled_train = np.ravel(shuffled_train , order = 'F')
flat_UBD_train = np.ravel(UBD_train , order = 'F')
flat_SBD_train = np.ravel(SBD_train , order = 'F')

n2 = np.mean(flat_dADAM_train)
n3 = np.mean(flat_dLSTM_train)
n4 = np.mean(flat_dEN_train)
n5 = np.mean(flat_dBI_train)
n6 = np.mean(flat_UBD_train)
n7 = np.mean(flat_shuffled_train)
n8 = np.mean(flat_SBD_train)

t2 = 1.96*(np.std(flat_dADAM_train)/(np.sqrt(flat_dADAM_train.shape[0])))
t3 = 1.96*(np.std(flat_dLSTM_train)/(np.sqrt(flat_dLSTM_train.shape[0])))
t4 = 1.96*(np.std(flat_dEN_train)/(np.sqrt(flat_dEN_train.shape[0])))
t5 = 1.96*(np.std(flat_dBI_train)/(np.sqrt(flat_dBI_train.shape[0])))
t6 = 1.96*(np.std(flat_UBD_train)/(np.sqrt(flat_UBD_train.shape[0])))
t7 = 1.96*(np.std(flat_shuffled_train)/(np.sqrt(flat_shuffled_train.shape[0])))
t8 = 1.96*(np.std(flat_SBD_train)/(np.sqrt(flat_SBD_train.shape[0])))

X = ['Train','Test']
fig = go.Figure()

fig.add_trace(go.Bar(x= X,
                y=np.array([n2,m2]),
                     error_y=dict(type='data', array= np.array([t2,s2])),
                name='FNN-ADAM',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x= X,
                y= np.array([n3,m3]),
                error_y=dict(type='data', array= np.array([t3,s3])),
                name='LSTM',
                marker_color='rgb(200,125,150)'
                ))
fig.add_trace(go.Bar(x=X,
                y= np.array([n4,m4]),
                error_y=dict(type='data', array= np.array([t4,s4])),
                name='ED-LSTM',
                marker_color='rgb(13, 255, 20)'
                ))
fig.add_trace(go.Bar(x=X,
                y= np.array([n5,m5]),
                error_y=dict(type='data', array= np.array([t5,s5])),
                name='BD-LSTM',
                marker_color='rgb(255, 10, 10)'
                ))


fig.update_layout(
    #title='Mean and Error of RMSE for 10 prediction horizons' ,
    xaxis_tickfont_size=20,
    width = 750,
    yaxis=dict(
        title='RMSE(Mean)',
        titlefont_size=20,
        tickfont_size=20,
    ),
    legend=dict(
        x=0.94,
        y=1.0,
        font = dict(size = 18),
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.2, # gap between bars of adjacent location coordinates.
    bargroupgap=0.05# gap between bars of the same location coordinate.
   
)
fig.show()

In [ ]:
X = ['Train','Test']
fig = go.Figure()
fig.add_trace(go.Bar(x=X,
                y= np.array([n5,m5]),
                error_y=dict(type='data', array= np.array([t5,s5])),
                name='BD-LSTM',
                marker_color='rgb(255, 10, 10)'
                ))
fig.add_trace(go.Bar(x= X,
                y=np.array([n7,m7]),
                     error_y=dict(type='data', array= np.array([t7,s7])),
                name='BD-LSTM*',
                marker_color='#FF851B'
                ))
fig.add_trace(go.Bar(x=X,
                y= np.array([n6,m6]),
                error_y=dict(type='data', array= np.array([t6,s6])),
                name='UBD-LSTM',
                marker_color='lightseagreen'
                ))
            
fig.add_trace(go.Bar(x=X,
                y= np.array([n8,m8]),
                error_y=dict(type='data', array= np.array([t8,s8])),
                name='SBD-LSTM',
                marker_color='darkblue'
                ))


fig.update_layout(
    #title='Mean and Error of RMSE for 10 prediction horizons' ,
    xaxis_tickfont_size=20,
    width = 700,
    yaxis=dict(
        title='RMSE(Mean)',
        titlefont_size=20,
        tickfont_size=20,
    ),
    legend=dict(
        x=0.94,
        y=1.0,
        font = dict(size = 18),
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.2, # gap between bars of adjacent location coordinates.
    bargroupgap=0.05# gap between bars of the same location coordinate.
   
)
fig.show()

Table for all time steps


In [ ]:
row_train_mean = [n2,n3,n4,n5,n7,n6,n8,t2,t3,t4,t5,t7,t6,t8]
row_test_mean = [m2,m3,m4,m5,m7,m6,m8,s2,s3,s4,s5,s7,s6,s8]
all_meth = [dADAM, dLSTM, dEN, dBI, shuffled, dBI_univ, dBI_season]
step1_mean = [np.mean(arr[:,0]) for arr in all_meth]
step1_err = [1.96*(np.std(arr[:,0])/(np.sqrt(arr[:,0].shape[0]))) for arr in all_meth]
step2_mean = [np.mean(arr[:,1]) for arr in all_meth]
step2_err = [1.96*(np.std(arr[:,1])/(np.sqrt(arr[:,1].shape[0]))) for arr in all_meth]
step3_mean = [np.mean(arr[:,2]) for arr in all_meth]
step3_err = [1.96*(np.std(arr[:,2])/(np.sqrt(arr[:,2].shape[0]))) for arr in all_meth]
step4_mean = [np.mean(arr[:,3]) for arr in all_meth]
step4_err = [1.96*(np.std(arr[:,3])/(np.sqrt(arr[:,3].shape[0]))) for arr in all_meth]
step5_mean = [np.mean(arr[:,4]) for arr in all_meth]
step5_err = [1.96*(np.std(arr[:,4])/(np.sqrt(arr[:,4].shape[0]))) for arr in all_meth]
step6_mean = [np.mean(arr[:,5]) for arr in all_meth]
step6_err = [1.96*(np.std(arr[:,5])/(np.sqrt(arr[:,5].shape[0]))) for arr in all_meth]
step7_mean = [np.mean(arr[:,6]) for arr in all_meth]
step7_err = [1.96*(np.std(arr[:,6])/(np.sqrt(arr[:,6].shape[0]))) for arr in all_meth]
step8_mean = [np.mean(arr[:,7]) for arr in all_meth]
step8_err = [1.96*(np.std(arr[:,7])/(np.sqrt(arr[:,7].shape[0]))) for arr in all_meth]
step9_mean = [np.mean(arr[:,8]) for arr in all_meth]
step9_err = [1.96*(np.std(arr[:,8])/(np.sqrt(arr[:,8].shape[0]))) for arr in all_meth]
step10_mean = [np.mean(arr[:,9]) for arr in all_meth]
step10_err = [1.96*(np.std(arr[:,9])/(np.sqrt(arr[:,9].shape[0]))) for arr in all_meth]
##print the corresponding list to get mean/error

1 months ahead prediction using Univariate UBD-LSTM model

ignore the upcoming models it was tried for one month ahead prediction

In [ ]:

def split_sequences_univariate(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        
        seq_x, seq_y = sequences[i:end_ix], sequences[end_ix-1:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def monthly(train_X,train_y,val_X,val_y,n_out,n_hours,n_features):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_hours,n_features))))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
    model.fit(train_X, train_y, epochs=1000, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle= False)
    return model

def predict_model(model_old,n_hours,n_features,n_out):
  new_model = Sequential()
  new_model.add(Bidirectional(LSTM(50, activation='relu', input_shape=(n_hours,n_features))))
  new_model.add(Dense(n_out))
  new_model.build(input_shape = (1,n_hours,n_features))
  old_weights = model_old.get_weights()
  new_model.set_weights(old_weights)
  new_model.compile(optimizer='adam', loss='mse')
  return new_model

dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,-7]
values_test=values_test[:,-7]
values_train = values_train.astype('float32')
values_test = values_test.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
concat = np.concatenate((values_train,values_test),axis = 0)
print(concat.shape)
maxv = max(concat)
minv = min(concat)
scaled = (concat-minv)/(maxv-minv)
scaled_train = scaled[:values_train.shape[0]]
scaled_test = scaled[values_train.shape[0]:]
time1 = time.time()
outfile_monthly = open('results/AnandVihar/monthly.txt','a+')
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            train_X, train_y = split_sequences_univariate(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences_univariate(scaled_test,n_hours,n_out)
            val_X = val_X.reshape(val_X.shape[0] , val_X.shape[1] , 1)
            train_X = train_X.reshape(train_X.shape[0] , train_X.shape[1] , 1)
            test_X, test_y = val_X[288:,:,:],val_y[288:,:]
            val_X, val_y = val_X[0:288,:,:],val_y[0:288,:]
            print("train_X.shape, train_y.shape, test_X.shape, test_y.shape=",train_X.shape, train_y.shape, test_X.shape, test_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = 1
            print("n_features = ",n_features)
            model_monthly= monthly(train_X,train_y,val_X,val_y,n_out,n_hours,n_features)
            values_list = []
            #creating the initial input from last 5 values of December which was distributed to test data
            #now this last 5 window will be used to create next 30 days of predictions (= 30*3 = 90 values) , 3 values for each day.
            ip = test_X[-1,:,:]
            initial_input = ip.reshape(1,ip.shape[0],1)
            print("initial_input=",initial_input.shape)
            new_model = predict_model(model_monthly,n_hours,n_features,n_out)
            for st in range(9):
              next_10 = new_model.predict(initial_input)
              values_list.extend(next_10)
              new_ip = next_10[:,5:]
              initial_input = new_ip.reshape(1,new_ip.shape[1],1)

            write_list = [inner for outer in values_list for inner in outer]
            final_write_list = [(maxv - minv)*item +minv for item in write_list]
            for item in final_write_list:
              item = round(item,4)
              outfile_monthly.write(str(item) + "  ")  

        outfile_monthly.write("\n")    

outfile_monthly.close()         
time2 = time.time()        
print("MODEL_time:",time2-time1)






Plot for one-month ahead with uncertainty

In [ ]:

from datetime import date, datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    if not isinstance(delta, timedelta):
        delta = timedelta(**delta)
    while current < end:
        yield current
        current += delta


start = datetime(2020,12,11)
end = datetime(2021,1,10)
time_axis = []
#this unlocks the following interface:
for dt in datetime_range(start, end, {'days': 0, 'hours':8}):
  time_axis.append(dt)

future_prediction = []
file1 = open('results/AnandVihar/monthly.txt','r')
Lines = file1.readlines()
for line in Lines:
  strline = line.strip()
  li = strline.split()
  li = li[:90]
  future_prediction.append([float(li[k]) for k in range(90)])

future_prediction = np.array(future_prediction)
print(future_prediction.shape)
pred_values = future_prediction.mean(axis=0)
error = np.array([1.96*(np.std(future_prediction[:,k])/(np.sqrt(future_prediction[:,k].shape[0]))) for k in range(90)]) 
per_05 = pred_values + error
per_95 = pred_values - error
fig = plt.figure()
params = {'legend.fontsize': 12}
plt.rcParams.update(params)
ax = fig.add_subplot(111) 
mu = ax.plot(time_axis, pred_values, color='blue', label = 'Mean')
s1 = ax.plot(time_axis, per_05, color='green', alpha=0.4,label = 'Error')
s2 = ax.plot(time_axis, per_95, color='green', alpha=0.4)
ax.fill_between(time_axis, per_05, per_95, facecolor='g', alpha=0.4)
ax.set_ylabel('PM2.5 Concentration') 
ax.set_xlabel('Time') 
plt.xticks(rotation = 45)
# plt.title('Predicted')
ax.legend()
#plt.savefig("/content/drive/My Drive/covid project/Model/Results/"+name+"/"+Mname+'/future_pred.png',dpi=300) 
plt.show()
plt.close()

Trace plot for month of September using seasonal model SBD-LSTM

In [ ]:

def trace_model(train_X,july_X,train_y,july_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu',input_shape=(n_hours,n_features))))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mse')
# fit model
    model.fit(train_X, train_y, epochs=50, batch_size=20, validation_data=(val_X, val_y), verbose=0, shuffle=False)

    july_pred = model.predict(july_X,verbose = 0)
    july_inv = ((maxv - minv) * july_pred  + minv)
    return july_inv


time1 = time.time()
dataset_train = read_csv('train_pollution.csv', header=0, index_col=0)
dataset_test = read_csv('test_pollution.csv', header=0, index_col=0)
values_train = dataset_train.values
values_test = dataset_test.values
values_train=values_train[:,1:-6]
values_test=values_test[:,1:-6]
values_train = values_train.astype('float32')
values_test = values_test.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
concat = np.concatenate((values_train,values_test),axis = 0)
maxv = max(concat[:,-1])
minv = min(concat[:,-1])
scaled = scaler.fit_transform(concat)
scaled_train = scaled[:values_train.shape[0],:]
scaled_test = scaled[values_train.shape[0]:,:]
allres = []
for i,k in enumerate(out_range):
    for j in range(1,31):
        for m in range(5,6):
            n_out=k
            n_hours=m
            print("Number of steps predicted= ",n_out,"Number of steps Multivariate threads ussed= ",n_hours, "iteration number= ",j)
            
            train_X, train_y = split_sequences(scaled_train,n_hours,n_out)
            val_X, val_y = split_sequences(scaled_test,n_hours,n_out)
            july_X, july_y = split_sequences(scaled_july,n_hours,n_out)
            print(val_X.shape,val_y.shape)
            print("july shape=",july_X.shape)
            #test_X, test_y = val_X[365:,:,:],val_y[365:,:]
            val_X, val_y = val_X[0:365,:,:],val_y[0:365,:]
            print("train_X.shape, train_y.shape =",train_X.shape, train_y.shape)
            print("val X, val y",val_X.shape, val_y.shape)
            n_features = train_X.shape[2]
            #july_X = test_X
            #july_y = test_y
            july_pred = trace_model(train_X,july_X,train_y,july_y,val_X,val_y,n_out,n_hours,n_features,minv,maxv)
            july_pred = np.array(july_pred)
            predicted = []
            for ctr in range(0,july_pred.shape[0],10):
              predicted.append(july_pred[ctr])
            predicted = np.array(predicted)  
            allres.append(predicted.flatten())
   
july_y = ((maxv - minv) * july_y  + minv)
actual = []
for ctr in range(0,july_y.shape[0],10):
  actual.append(july_y[ctr])
actual = np.array(actual).flatten()
allres = np.array(allres)
final_pred_y = np.mean(allres, axis = 0)

time2 = time.time()
print("MODEL_july:",time2-time1)

## Plotting


Corresponding Trace Plot

In [ ]:
x_data=np.linspace(0,final_pred_y.shape[0], num=final_pred_y.shape[0])
error = np.array([1.96*(np.std(allres[:,k])/(np.sqrt(allres[:,k].shape[0]))) for k in range(allres.shape[1])]) 
per_05 = final_pred_y + error
per_95 = final_pred_y - error
fig = plt.figure()
params = {'legend.fontsize': 10}
plt.rcParams.update(params)
ax = fig.add_subplot(111) 
act = ax.plot(x_data, actual,color = 'red', label='actual')
pr = ax.plot(x_data, final_pred_y,color = 'blue', label='predicted')
s1 = ax.plot(x_data, per_05, color='green', alpha=0.4,label = 'Error')
s2 = ax.plot(x_data, per_95, color='green', alpha=0.4)
ax.fill_between(x_data, per_05, per_95, facecolor='g', alpha=0.4)
ax.set_ylabel('PM2.5 Concentration')
ax.set_xlabel('Time (samples)') 
ax.set_title('Actual vs Predicted')
ax.legend()
plt.show()
plt.close()